In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('node-edges.xlsx', header=None, )
df.columns = df.iloc[61]
df = df.iloc[0:30]

count = 0
for i in range(0, len(df.columns)):
    if (i % 2 != 0):
        df.columns.values[i] = ("%02d" % count) + ': Destination'
        count += 1
        
slices = []
for i in range(2, len(df.columns), 2):
    s = df.iloc[:, i-2:i]
    s = s.dropna()
    s = s[s.iloc[:,1] != '-'] # edges are marked at entrance AND destination, so missing values can be removed
    slices.append(s)
    
nodes = {}
edges = {}
for s in slices:
    node_name = s.iloc[:,0].name
    node_name = str.strip(node_name.split(':')[1]) # remove number and whitespace from name
    edges.update({e: node_name for e in s.iloc[:,0]})
    nodes.update({node_name: {'node_name': node_name,
                              'transitions': [{'entrance': e, 'destination': d} for e, d in s.iloc],
                              'visited':  False,
                              'previous': {}}})
    
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
        
graph = Graph(nodes, edges)

In [2]:
import copy

def graph_algo(src, dest, graph):
    if src == dest:
        return True
    src_node = graph.nodes[src]
    src_node['visited'] = True
    queue = [src_node]

    while len(queue) > 0:
        # print('Queue: ', sorted([x['node_name'] for x in queue]))
        current = queue.pop(0)
        # if current['node_name'] == dest:
            # print('Current: %s equals Dest: %s' % (current['node_name'], dest))
            # print('### FOUND IT ####')
            # print(current)
        for t in current['transitions']:
            node_name = graph.edges.get(t['destination'])
            if node_name:
                node = graph.nodes[node_name]
                # add previous
                prev = node['previous']
                if prev.get(current['node_name']):
                    prev[current['node_name']].append(t)
                else:
                    prev[current['node_name']] = [t]
                # mark visited
                if not node['visited']:
                    node['visited'] = True
                    queue.append(node)
    return graph.nodes[dest]

graph_copy = copy.deepcopy(graph)
dest = graph_algo('Slateport/Mauville/Verdanturf', 'Rustboro', graph_copy)
dest

{'node_name': 'Rustboro',
 'transitions': [{'entrance': 'Rustboro Mart',
   'destination': 'Petalburg Woods SE'},
  {'entrance': 'Rustboro Pokecenter',
   'destination': 'Sootopolis Pokecenter S'},
  {'entrance': 'Rustboro SE', 'destination': 'Meteor Falls 1F E'},
  {'entrance': 'Rustboro Cut House', 'destination': 'Slateport Pokecenter S'}],
 'visited': True,
 'previous': {'Pokecenters': [{'entrance': 'Slateport Pokecenter S',
    'destination': 'Rustboro Cut House'},
   {'entrance': 'Sootopolis Pokecenter S',
    'destination': 'Rustboro Pokecenter'}],
  'Meteor Falls': [{'entrance': 'Meteor Falls 1F E',
    'destination': 'Rustboro SE'}],
  'Petalburg Woods': [{'entrance': 'Petalburg Woods SE',
    'destination': 'Rustboro Mart'}]}}

In [6]:
reachable = set()

def reach(node_name, graph, depth_max):
    node = graph.nodes[node_name]
    prev = node['previous'].keys()
    # print(prev)
    for k in prev:
        reachable.add(k)
        if depth_max > 0:
            reach(k, graph, depth_max-1)
reach('Rustboro', graph_copy, 5)
len(reachable)

28

In [124]:
name = dest['node_name']
the_node = graph_copy.nodes[name]
prev = the_node['previous']
flat_prev = [[(node[0], trans)] for node in list(prev.items())
    for trans in node[1]]

def list_prev(node_name):
    node = graph_copy.nodes[node_name]
    flat_prev = [(node[0], trans) for node in list(node['previous'].items())
                                    for trans in node[1]]
    
    
flat_prev[0]

[('Pokecenters',
  {'entrance': 'Slateport Pokecenter S', 'destination': 'Rustboro Cut House'})]

In [139]:
def flatten_prev(node_name, graph):
    node = graph.nodes[node_name]
    return [(node[0], trans) for node in list(node['previous'].items())
                        for trans in node[1]]
def list_prev(node_name, graph):
    node = graph.nodes[node_name]
    flat_prev = flatten_prev(node_name, graph)
    return flat_prev
    

print(flatten_prev('Slateport/Mauville/Verdanturf', graph_copy)[0])
print(flatten_prev('Aqua Hideout', graph_copy)[0])
list_prev('Slateport/Mauville/Verdanturf', graph_copy)

('Aqua Hideout', {'entrance': 'Aqua Hideout B2F NW', 'destination': 'Slateport Contest'})
('Slateport/Mauville/Verdanturf', {'entrance': 'Slateport Contest', 'destination': 'Aqua Hideout B2F NW'})


[('Aqua Hideout',
  {'entrance': 'Aqua Hideout B2F NW', 'destination': 'Slateport Contest'}),
 ('Pokecenters',
  {'entrance': 'Rustboro Pokecenter S', 'destination': 'Mauville S'}),
 ('Pokecenters',
  {'entrance': 'Fallarbor Pokecenter W', 'destination': 'R110 Cycle Road S'}),
 ('Pokecenters',
  {'entrance': 'Lilycove Pokecenter W', 'destination': 'Slateport Mart'}),
 ('Mt Pyre',
  {'entrance': 'Mt Pyre 2F Ceiling 1', 'destination': 'Slateport Museum'}),
 ('Mt Pyre', {'entrance': 'Mt Pyre 3F Up', 'destination': 'Mauville Gym'}),
 ('E4 Pokecenter',
  {'entrance': 'Lavaridge Pokecenter S',
   'destination': 'Slateport Pokecenter'}),
 ('Oldale/Petalburg',
  {'entrance': 'Oldale SE', 'destination': 'Slateport Museum'}),
 ('Oldale/Petalburg',
  {'entrance': 'Petalburg Gym', 'destination': 'Slateport SE'}),
 ('Fallarbor/Lavaridge',
  {'entrance': 'Fallarbor Move Relearner', 'destination': 'Verdanturf SW'}),
 ('Fallarbor/Lavaridge',
  {'entrance': 'R114 Lanette', 'destination': 'R109 Seashore

In [90]:
graph_copy.nodes['Pokecenters']['previous']

{'Slateport/Mauville/Verdanturf': [{'entrance': 'Slateport Mart',
   'destination': 'Lilycove Pokecenter W'}]}

In [91]:
graph_copy.nodes['Slateport/Mauville/Verdanturf']

{'node_name': 'Slateport/Mauville/Verdanturf',
 'transitions': [{'entrance': 'Slateport Contest',
   'destination': 'Aqua Hideout B2F NW'},
  {'entrance': 'Slateport Fan Club', 'destination': 'Strength'},
  {'entrance': 'Slateport Mart', 'destination': 'Lilycove Pokecenter W'},
  {'entrance': 'Slateport Museum', 'destination': 'Mt Pyre 2F Ceiling 1'},
  {'entrance': 'Slateport Name Rater', 'destination': 'Dive'},
  {'entrance': 'Slateport Pokecenter',
   'destination': 'Lavaridge Pokecenter S'},
  {'entrance': 'Slateport SE', 'destination': 'Petalburg Gym'},
  {'entrance': 'R109 Seashore House', 'destination': 'R114 Lanette'},
  {'entrance': 'R110 Cycle Road S', 'destination': 'Fallarbor Pokecenter W'},
  {'entrance': 'Daycare', 'destination': 'Dead-End'},
  {'entrance': 'Mauville Bike Shop', 'destination': 'Dead-End'},
  {'entrance': 'Mauville Casino', 'destination': 'Dead-End'},
  {'entrance': 'Mauville Gym', 'destination': 'Mt Pyre 3F Up'},
  {'entrance': 'Mauville Mart', 'destinati

In [35]:
list(edges.keys())
len(list(nodes.keys()))
sorted(list(nodes.keys()))
# nodes

['Abandoned Ship',
 'Aqua Hideout',
 'Dewford',
 'Dive Hub',
 'E4 Pokecenter',
 'Fallarbor/Lavaridge',
 'Fortree/Lilycove',
 'Granite Cave',
 'Jagged Pass',
 'Lilycove Contest Hall',
 'Lilycove Department Store',
 'Magma Hideout',
 'Meteor Falls',
 'Mirage Tower',
 'Mossdeep',
 'Mt Pyre',
 'Navel Rock',
 'Oldale/Petalburg',
 'Pacifidlog',
 'Petalburg Woods',
 'Places of Interest',
 'Pokecenters',
 'Route 119/123',
 'Rustboro',
 'Rusturf Tunnel',
 'Seafloor Cavern',
 'Sky Pillar',
 'Slateport/Mauville/Verdanturf',
 'Sootopolis',
 'Surf Hub',
 'Victory Road',
 'Waterfall Hub']

In [34]:
sorted(list(edges.keys()))

['Abandoned Ship 1F Left Exit',
 'Abandoned Ship 1F Left N',
 'Abandoned Ship 1F Left S',
 'Abandoned Ship 1F Left Stairs',
 'Abandoned Ship 1F Right Exit',
 'Abandoned Ship 1F Right SW',
 'Abandoned Ship B1F SW',
 'Abandoned Ship Rooms NE',
 'Abandoned Ship Rooms SE',
 'Aqua Hideout B1F NE Room N',
 'Aqua Hideout B1F NE Room SE',
 'Aqua Hideout B1F NE Room SW',
 'Aqua Hideout B1F S Room E',
 'Aqua Hideout B1F S Room NW',
 'Aqua Hideout B1F S Room SW',
 'Aqua Hideout B2F NW',
 'Aqua Hideout B2F SW',
 'Aqua Hideout Warps 1 N',
 'Aqua Hideout Warps 1 S',
 'Aqua Hideout Warps 1 SE',
 'Aqua Hideout Warps 1 SW',
 'Aqua Hideout Warps 2 C',
 'Aqua Hideout Warps 2 E',
 'Aqua Hideout Warps 2 W',
 'Aqua Hideout Warps 3 C',
 'Aqua Hideout Warps 3 W',
 'Aqua Hideout Warps 4 C',
 'Aqua Hideout Warps 4 E',
 'Aqua Hideout Warps 4 W',
 'Contest Hall NW',
 'Contest Hall S',
 'Department Store 1F Exit',
 'Department Store 2F Up',
 'Department Store 3F Down',
 'Department Store 3F Up',
 'Department Store

In [54]:
[e['destination'] for e in nodes['Oldale/Petalburg']['transitions']]

['Aqua Hideout B1F S Room NW',
 'Slateport Museum',
 'Slateport SE',
 'Rusturf Tunnel S',
 'Pacifidlog NW',
 'Aqua Hideout B1F NE Room N']

In [52]:
nodes['Oldale/Petalburg']['transitions']

[{'entrance': 'Oldale Pokecenter',
  'destination': 'Aqua Hideout B1F S Room NW'},
 {'entrance': 'Oldale SE', 'destination': 'Slateport Museum'},
 {'entrance': 'Petalburg Gym', 'destination': 'Slateport SE'},
 {'entrance': 'Petalburg Mart', 'destination': 'Rusturf Tunnel S'},
 {'entrance': 'Petalburg SE', 'destination': 'Pacifidlog NW'},
 {'entrance': 'Petalburg SW', 'destination': 'Aqua Hideout B1F NE Room N'}]

In [52]:
lol = ['a', 'b', 'c']
lol.pop(0)
lol

['b', 'c']